## Order TDFP

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace
from tqdm import tqdm
from functools import lru_cache
import warnings
warnings.simplefilter("ignore")
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
lp = {"l1" :(0,1),
      "l2" :(1,(1+3)),
      "l4" :((1+3),(1+3+3)),
      "l3" :((1+3+3),(1+3+3+10)),
      "l5" :((1+3+10+3),(1+3+10+3+7)),
      "l6" :((1+3+10+3+7),(1+3+10+3+7+9)),
      "l7" :((1+3+10+3+7+9),(1+3+10+3+7+9+21)),
      "l8" :((1+3+10+3+7+9+21),(1+3+10+3+7+9+21+30)),
      "l10":((1+3+10+3+7+9+21+30),(1+3+10+3+7+9+21+30+3049)),
      "l9" :((1+3+10+3+7+9+21+30+3049),(1+3+10+3+7+9+21+30+3049+70)),
      "l11":((1+3+10+3+7+9+21+30+70+3049),(1+3+10+3+7+9+21+30+70+3049+9147)),
      "l12":((1+3+10+3+7+9+21+30+70+3049+9147),(1+3+10+3+7+9+21+30+70+3049+9147+30490))}
lvls = []
#n = 1
for i in lp.keys():
    a,b = lp[i]
    lvls.append([i[1:]]*28*(b-a))
   # n+=1
levels = pd.concat([pd.DataFrame({'levels':lvls[i]}) for i in range(len(lvls))]).reset_index(drop=True)

def tdfp(df, col2: str, col1='unique_id', col3='ds'):
    states = ['CA', 'TX', 'WI']
    categories = ['HOBBIES', 'HOUSEHOLD', 'FOODS']
    df_ = df.copy()
    splits = df_[col1].str.rsplit("/", n=1, expand=True)
    df_["parent_id"] = splits[0]
    df_["child_id"] = splits[1].fillna('')
    df_['group_sum'] = df_.groupby([col3, "parent_id", "levels"])[col2].transform("sum")
    df_["prop_to_parent"] = df_[col2] / df_['group_sum']
    for subset in [states, categories]:
        cond = (df_["parent_id"] == 'TOTAL') & (df_["child_id"].isin(subset))
        group_sum = df_.loc[cond].groupby([col3, "parent_id", "levels"])[col2].transform("sum")
        df_.loc[cond, 'group_sum'] = group_sum
        df_.loc[cond, "prop_to_parent"] = df_.loc[cond, col2] / group_sum
    condition = (df_["parent_id"] == 'TOTAL') & (df_["child_id"] == '')
    df_.loc[condition, 'group_sum'] = df_.loc[condition, col2]
    df_.loc[condition, "prop_to_parent"] = 1.0
    return df_

def TDFP(df, col, ts_id):
    proportion = []
    df = pd.concat([ts_id, df], axis = 1)
    for ds in tqdm(range(1914, 1942)):
        df2 = tdfp(df[df['ds'] == ds], col2 = col)
        total = df2[(df2['levels']=='1')&(df2['ds']==ds)][col].iloc[0]
        #print(total)
        df_ds = df2[df2['ds'] == ds].copy()

        parent_map = dict(zip(df_ds["unique_id"], df_ds["parent_id"]))
        prop_to_parent = dict(zip(df_ds["unique_id"], df_ds["prop_to_parent"]))
        @lru_cache(maxsize=None)
        def get_global_prop(node):
            if parent_map[node] == node:  
                return 1.0
            return get_global_prop(parent_map[node]) * prop_to_parent[node]

        df_ds["prop_global"] = df_ds["unique_id"].map(get_global_prop)
        df_ds[f"tdfp_{col}"] = df_ds["prop_global"] * total
        proportion.append(df_ds)
        get_global_prop.cache_clear()
    return pd.concat(proportion).sort_index()

test    = pd.read_pickle(f"{path1}future_28.pkl").reset_index(drop=True)
tags    = pd.read_pickle(f"{path1}tags.bin")
ts      = pd.concat([levels, test[['unique_id','ds']]],axis = 1)

c:\py\Anaconda3\lib\site-packages\numba\np\ufunc\parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12040. The TBB threading layer is disabled.
  warnings.warn(problem)


### TDFP L1

In [14]:
needs = ['levels','unique_id','ds','name','ot_90','ot_95','ot_99']
cols  = ['ot_90','ot_95','ot_99']
names = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu', 'ets_td', 'ets_mint']
#names = ['lgb_base','lgb_td', 'ets_base', 'ets_td']
lt = 1 # lead time
lgbs = pd.read_pickle(f"lgbInvtSim_L{lt}.pkl")
etss = pd.read_pickle(f"etsInvtSim_L{lt}.pkl")
df = pd.concat([lgbs,etss])
tdfpo90,tdfpo95,tdfpo99 = [], [], []
for j in names:
    a = TDFP(df[df['name'] == j], col = 'ot_90',ts_id = ts)
    tdfpo90.append(a)
    b = TDFP(df[df['name'] == j], col = 'ot_95',ts_id = ts)
    tdfpo95.append(b)
    c = TDFP(df[df['name'] == j], col = 'ot_99',ts_id = ts)
    tdfpo99.append(c)
a_ = pd.concat(tdfpo99)
b_ = pd.concat(tdfpo95)
c_ = pd.concat(tdfpo90)
Order_TDFP = pd.concat([c_[['name','tdfp_ot_90']], b_['tdfp_ot_95'], a_['tdfp_ot_99']],axis = 1)
Order_TDFP.to_pickle(f"OrderTDFP_L{lt}.pkl")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:08<00:00,  3.18it/s]


### TDFP L3

In [15]:
needs = ['levels','unique_id','ds','name','ot_90','ot_95','ot_99']
cols  = ['ot_90','ot_95','ot_99']
names = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu', 'ets_td', 'ets_mint']
#names = ['lgb_base','lgb_td', 'ets_base', 'ets_td']
lt = 3 # lead time
lgbs = pd.read_pickle(f"lgbInvtSim_L{lt}.pkl")
etss = pd.read_pickle(f"etsInvtSim_L{lt}.pkl")
df = pd.concat([lgbs,etss])
tdfpo90,tdfpo95,tdfpo99 = [], [], []
for j in names:
    a = TDFP(df[df['name'] == j], col = 'ot_90',ts_id = ts)
    tdfpo90.append(a)
    b = TDFP(df[df['name'] == j], col = 'ot_95',ts_id = ts)
    tdfpo95.append(b)
    c = TDFP(df[df['name'] == j], col = 'ot_99',ts_id = ts)
    tdfpo99.append(c)
a_ = pd.concat(tdfpo99)
b_ = pd.concat(tdfpo95)
c_ = pd.concat(tdfpo90)
Order_TDFP = pd.concat([c_[['name','tdfp_ot_90']], b_['tdfp_ot_95'], a_['tdfp_ot_99']],axis = 1)
Order_TDFP.to_pickle(f"OrderTDFP_L{lt}.pkl")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:08<00:00,  3.26it/s]


### TDFP L7

In [16]:
needs = ['levels','unique_id','ds','name','ot_90','ot_95','ot_99']
cols  = ['ot_90','ot_95','ot_99']
names = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu', 'ets_td', 'ets_mint']
#names = ['lgb_base','lgb_td', 'ets_base', 'ets_td']
lt = 7 # lead time
lgbs = pd.read_pickle(f"lgbInvtSim_L{lt}.pkl")
etss = pd.read_pickle(f"etsInvtSim_L{lt}.pkl")
df = pd.concat([lgbs,etss])
tdfpo90,tdfpo95,tdfpo99 = [], [], []
for j in names:
    a = TDFP(df[df['name'] == j], col = 'ot_90',ts_id = ts)
    tdfpo90.append(a)
    b = TDFP(df[df['name'] == j], col = 'ot_95',ts_id = ts)
    tdfpo95.append(b)
    c = TDFP(df[df['name'] == j], col = 'ot_99',ts_id = ts)
    tdfpo99.append(c)
a_ = pd.concat(tdfpo99)
b_ = pd.concat(tdfpo95)
c_ = pd.concat(tdfpo90)
Order_TDFP = pd.concat([c_[['name','tdfp_ot_90']], b_['tdfp_ot_95'], a_['tdfp_ot_99']],axis = 1)
Order_TDFP.to_pickle(f"OrderTDFP_L{lt}.pkl")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:10<00:00,  2.65it/s]


### TDFP L14

In [17]:
needs = ['levels','unique_id','ds','name','ot_90','ot_95','ot_99']
cols  = ['ot_90','ot_95','ot_99']
names = ['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint', 'ets_base', 'ets_bu', 'ets_td', 'ets_mint']
#names = ['lgb_base','lgb_td', 'ets_base', 'ets_td']
lt = 14 # lead time
lgbs = pd.read_pickle(f"lgbInvtSim_L{lt}.pkl")
etss = pd.read_pickle(f"etsInvtSim_L{lt}.pkl")
df = pd.concat([lgbs,etss])
tdfpo90,tdfpo95,tdfpo99 = [], [], []
for j in names:
    a = TDFP(df[df['name'] == j], col = 'ot_90',ts_id = ts)
    tdfpo90.append(a)
    b = TDFP(df[df['name'] == j], col = 'ot_95',ts_id = ts)
    tdfpo95.append(b)
    c = TDFP(df[df['name'] == j], col = 'ot_99',ts_id = ts)
    tdfpo99.append(c)
a_ = pd.concat(tdfpo99)
b_ = pd.concat(tdfpo95)
c_ = pd.concat(tdfpo90)
Order_TDFP = pd.concat([c_[['name','tdfp_ot_90']], b_['tdfp_ot_95'], a_['tdfp_ot_99']],axis = 1)
Order_TDFP.to_pickle(f"OrderTDFP_L{lt}.pkl")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:08<00:00,  3.32it/s]


In [13]:
Order_TDFP#['name']#.unique()

,name,tdfp_ot_90,tdfp_ot_95,tdfp_ot_99
0,lgb_base,0.000000,0.000000,0.000000
1,lgb_base,28765.856606,29343.083450,30425.865012
2,lgb_base,40407.312866,40407.312866,40407.312866
3,lgb_base,40387.682971,40387.682971,40387.682971
4,lgb_base,41906.803868,41906.803868,41906.803868
...,...,...,...,...
1199515,ets_mint,NaN,NaN,NaN
1199516,ets_mint,NaN,NaN,NaN
1199517,ets_mint,NaN,NaN,NaN
1199518,ets_mint,NaN,NaN,NaN


## Fixed Order Simulation:

In [1]:
import warnings
warnings.simplefilter("ignore")
#path1  = 'D:/yjz/Extension for hts/JayCode/Models/'
from Invtsim import *
import numpy as np
import pandas as pd
from scipy.stats import norm
from tqdm import tqdm
def loop2( truth, NAME:str, fcst, residual = 0,
         gap1: int = 28, gap2:int = 1913,
         n = None, 
         df_list:list = [], 
         L_ = 3):
    if n == None:
        n = int(len(truth)/gap1)
    for i in tqdm(range(n)):
        fcst_1  = fcst[i*gap1:(i+1)*gap1]
        truth_1 = truth[i*gap1:(i+1)*gap1]
        #res_1   = residual[i*gap2:(i+1)*gap2]
        df = InvtSim(fcst = 0, truth = truth_1, residual = residual, name = NAME, L=L_)##################
        res = df.ob_all_t_fixedcase(fixed_order = fcst_1)
        df_list.append(res.reset_index(drop = True))
    print("-----------DOWN!------------")
    return pd.concat(df_list).reset_index(drop=True)

## TD-FIXED SIMULATION:

### L1

In [2]:
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
TD   = pd.read_pickle("OrderTDFP.pkl")
truth= pd.read_pickle(f"{path1}future_28.pkl")['y']
Invt_df = []
names   = [ "lgb_base", "lgb_bu", "lgb_td", "lgb_mint", "ets_base", "ets_bu", "ets_td", "ets_mint"]
lead_time = 1
for i in names:
    df = loop2(fcst = TD[TD['name'] == i].iloc[:,-3:], truth = truth, residual = [], NAME = i, df_list = [], L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"TDFPOrder_L{lead_time}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:06<00:00, 647.41it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:04<00:00, 664.44it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:04<00:00, 661.09it/s]


...

-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:05<00:00, 658.44it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:04<00:00, 660.44it/s]


-----------DOWN!------------


### L3

In [3]:
lead_time = 3
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
TD   = pd.read_pickle(F"OrderTDFP_L{lead_time}.pkl")
truth= pd.read_pickle(f"{path1}future_28.pkl")['y']
Invt_df = []
names   = [ "lgb_base", "lgb_bu", "lgb_td", "lgb_mint", "ets_base", "ets_bu", "ets_td", "ets_mint"]

for i in names:
    df = loop2(fcst = TD[TD['name'] == i].iloc[:,-3:], truth = truth, residual = [], NAME = i, df_list = [], L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"TDFPOrder_L{lead_time}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:18<00:00, 548.98it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:09<00:00, 616.54it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:09<00:00, 613.70it/s]


...

-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:09<00:00, 613.32it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:09<00:00, 613.99it/s]


-----------DOWN!------------


### L7

In [4]:
lead_time = 7
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
TD   = pd.read_pickle(F"OrderTDFP_L{lead_time}.pkl")
truth= pd.read_pickle(f"{path1}future_28.pkl")['y']
Invt_df = []
names   = [ "lgb_base", "lgb_bu", "lgb_td", "lgb_mint", "ets_base", "ets_bu", "ets_td", "ets_mint"]

for i in names:
    df = loop2(fcst = TD[TD['name'] == i].iloc[:,-3:], truth = truth, residual = [], NAME = i, df_list = [], L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"TDFPOrder_L{lead_time}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:26<00:00, 494.25it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:14<00:00, 571.37it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:26<00:00, 493.29it/s]


...

-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:15<00:00, 564.34it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:14<00:00, 574.05it/s]


-----------DOWN!------------


### L14

In [5]:
lead_time = 14
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
TD   = pd.read_pickle(F"OrderTDFP_L{lead_time}.pkl")
truth= pd.read_pickle(f"{path1}future_28.pkl")['y']
Invt_df = []
names   = [ "lgb_base", "lgb_bu", "lgb_td", "lgb_mint", "ets_base", "ets_bu", "ets_td", "ets_mint"]

for i in names:
    df = loop2(fcst = TD[TD['name'] == i].iloc[:,-3:], truth = truth, residual = [], NAME = i, df_list = [], L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"TDFPOrder_L{lead_time}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:25<00:00, 498.37it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:26<00:00, 495.20it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:35<00:00, 448.59it/s]


...

-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:26<00:00, 495.82it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:26<00:00, 495.61it/s]


-----------DOWN!------------
